In [ ]:
import os
import numpy as np
import sys
from PIL import Image
import PIL.ImageOps

%config Completer.use_jedi = False

# from np bitmap

https://medium.com/tensorflow/train-on-google-colab-and-run-on-the-browser-a-case-study-8a45f9b1474e

data from: 
https://github.com/googlecreativelab/quickdraw-dataset#get-the-data

In [ ]:
import time
file = '../../../../Downloads/full_numpy_bitmap_sheep.npy'
class_name, ext = os.path.splitext(os.path.basename(file))

sheep = np.load(file)

In [ ]:
from matplotlib.pyplot import imshow
from IPython.display import display, clear_output
count = 0
width=28
height=28
zoom=1
for s in sheep: 
    clear_output(wait=True)
    count += 1
    a = s.reshape(28,28)

    f_img = Image.fromarray(a)
    img = PIL.ImageOps.invert(f_img)
    display(img.resize((zoom*width, zoom*height), Image.BICUBIC, reducing_gap=3))
    time.sleep(.5)
    if count == 20: 
        break


In [ ]:
from matplotlib.pyplot import imshow


img = imshow(a)
img

## data from quickdraw lib
https://quickdraw.readthedocs.io/en/latest/index.html


RGB to grey

https://stackoverflow.com/questions/41966514/how-fast-in-python-change-3-channel-rgb-color-image-to-1-channel-gray


In [ ]:
import quickdraw as qd

qd_data = qd.QuickDrawData()

In [ ]:
qd_data.drawing_names

In [ ]:
sheep = qd.QuickDrawDataGroup('sheep', recognized=True)

In [ ]:
sheep.drawing_count

In [ ]:
sheep1 = sheep.get_drawing(0)
img = sheep1.image

img
img = img.crop()
img
a = np.array(img)
a.shape

In [ ]:
count = 0
width=28
height=28
zoom=1
for idx in list(range(sheep.drawing_count)): 

    clear_output(wait=True)
    count += 1
    
    drawing = sheep.get_drawing(idx)
    img = drawing.image
    if False: 
        a = s.reshape(28,28)

        f_img = Image.fromarray(a)
        img = PIL.ImageOps.invert(f_img)
        display(img.resize((zoom*width, zoom*height), Image.BICUBIC, reducing_gap=3))
    
    display(img)
    time.sleep(.5)
    if count == 20: 
        break


## data from quickdraw10 lib

https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/QuickDraw10.ipynb#scrollTo=6o30ipBPAQ5Y


## ndjson to svg
https://pypi.org/project/ndjsonTosvg/

https://github.com/thompson318/ndjsonTosvg/blob/master/ndjsontosvg/ui/ndjsontosvg_command_line.py


## ...

In [ ]:

import os
import numpy as np
import urllib.request

from random import sample, seed
from shutil import rmtree
from PIL import Image


from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator





data_path = '../data/'
npy_path = os.path.join(data_path, 'npy/')
train_path = os.path.join(data_path, 'train/')
test_path = os.path.join(data_path, 'test/')

for d in [data_path, npy_path, train_path, test_path]: 
    if not os.path.exists(d): 
        os.mkdir(d)
    
def download(class_name, target_path=npy_path):
    # https://medium.com/tensorflow/train-on-google-colab-and-run-on-the-browser-a-case-study-8a45f9b1474e
    base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
    cls_url = class_name.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    path, _ = urllib.request.urlretrieve(path, os.path.join(npy_path, f'{class_name}.npy'))
    return path


def save_pngs(a, path): 
    if not os.path.exists(path): 
        os.mkdir(path)
    else: 
        rmtree(path)
        os.mkdir(path)

    for idx, aa in enumerate(a): 
        f_img = Image.fromarray(aa.reshape(28,28))
        f_img.save(os.path.join(path, f'{idx}.png'))


def reshape_1d(a_1d, class_name): 
    x_train_4d = a_1d.reshape(a_1d.shape[0], 28,28,1)

    labels = np.full(a_1d.shape[0], class_name)



    #x_train_4d = x_train.reshape(x_train.shape[0], 28,28,1)
    #x_train_comb = np.concatenate((x_train_comb, x_train_4d), axis=0) 

    #labels = np.full(x_train.shape[0], cl)
    #y_train = np.append(y, labels)

    return x_train_4d, labels


def to_img(array_4d): 
    fft_p = array_4d.reshape(28,28)
    new_p = Image.fromarray(fft_p)
    if new_p.mode != 'RGB':
        new_p = new_p.convert('RGB')
    return new_p

def prepare_imgs_for_generator(classes, test_ratio, max_imgs_per_class, mode='array'): 
    seed(999)
    train_size = None
    
    if mode not in ['array', 'dir']: 
        raise ValueError


    x_train = np.empty([0, 28,28, 1])
    y_train = np.empty([0])
    x_test = np.empty([0, 28,28, 1])
    y_test = np.empty([0])
    class_names = {}
    
    for cl_idx, cl in enumerate(classes):
        print('*'*100)
        print(f'processing class **{cl}**' )
        file_name = f'{cl}.npy'
        source_path = npy_path

        if file_name not in os.listdir(source_path): 
            print(f'download dara to {source_path}')
            tmp_path = download(cl, source_path)
        else: 
            print(f'loading data from buffer: {source_path}')
            tmp_path = os.path.join(source_path, file_name)
        x = np.load(tmp_path)
        print('Shape of raw data: ',x.shape)

        if max_imgs_per_class is not None: 
            rand_idx = sample([i for i in range(x.shape[0])], max_imgs_per_class)
            x = x[rand_idx, :]

        # stratify: memorize abs train size of first class
        if train_size is None:    
            train_size =  int(x.shape[0]*(1-test_ratio))

        # split array into train and test
        x_train_1d = x[0:train_size, :]
        x_test_1d = x[train_size:x.shape[0], :]
        print(f'Shape after sampling, shuffle and first split: train: {x_train_1d.shape} | test: {x_test_1d.shape}')


        if mode == 'dir': 
            # save pngs to disk
            save_pngs(x_train_1d, os.path.join(train_path, cl))
            save_pngs(x_test_1d, os.path.join(test_path, cl))
            
            return 'ok'
        
        elif mode == 'array': 
            
            x_train_tmp, y_train_tmp = reshape_1d(x_train_1d, cl_idx)            
            x_train = np.concatenate((x_train, x_train_tmp), axis=0) 
            y_train = np.append(y_train, y_train_tmp)
            
            x_test_tmp, y_test_tmp = reshape_1d(x_test_1d, cl_idx)
            x_test = np.concatenate((x_test, x_test_tmp), axis=0) 
            y_test = np.append(y_test, y_test_tmp)
            
            class_names[cl_idx] = cl

            
    if mode == 'dir': 
        return 'ok'
    elif mode == 'array': 
        seed(22)
        rand_idx = sample([i for i in range(x_train.shape[0])], x_train.shape[0])
        
        num_classes=len(classes)
        y_train = to_categorical(y_train, num_classes)
        y_test = to_categorical(y_test, num_classes)
        
        return x_train[rand_idx], y_train[rand_idx], x_test, y_test, class_names


def build_set_generators(classes, max_imgs_per_class=10000, vali_ratio=.2, test_ratio=.2, batch_size=32):

    x_train, y_train, x_test, y_test, class_names = prepare_imgs_for_generator(
        classes=classes,
        test_ratio=test_ratio,
        max_imgs_per_class = max_imgs_per_class
    )

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=vali_ratio) # set validation split


    train_generator = train_datagen.flow(
        x=x_train, 
        y=y_train,
        shuffle=False,
        batch_size=batch_size,
        subset='training'
    ) # set as training data

    validation_generator = train_datagen.flow(
        x=x_train, 
        y=y_train,
        shuffle=False,
        batch_size=batch_size,
        subset='validation') # set as validation data

    test_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.0) # set validation split

    test_generator = test_datagen.flow(
        x=x_test, 
        y=y_test,
        shuffle=False,
        batch_size=batch_size
    ) # set as validation data
    
    return train_generator, validation_generator, test_generator, class_names


            
big_mamals = [
    "camel", "cow", "elephant", "giraffe", "horse", 
    "kangoroo", "lion", "panda", "rhinoceros", "tiger", "zebra"
]
n_classes = 3
class_list = big_mamals[0:n_classes]

train_generator, validation_generator, test_generator, class_names = build_set_generators(
    classes=class_list, 
    max_imgs_per_class=10000, 
    vali_ratio=.2, 
    test_ratio=.2, 
    batch_size=32
)

In [ ]:
from keras.models import Sequential, Model
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras import layers

def define_model(): 
    # Initialising the CNN
    classifier = Sequential()

    # Step 1 - Convolution
    classifier.add(Conv2D(32, (3, 3), input_shape = (28, 28, 1), activation = 'relu'))

    # Step 2 - Pooling
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Adding a second convolutional layer
    classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    # Step 3 - Flattening
    classifier.add(Flatten())

    # Step 4 - Full connection
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dense(units = 3, activation = 'sigmoid')) 
    
    classifier.compile(optimizer='rmsprop',  
          loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier


model = define_model()


def define_model2(input_shape=x_train.shape[1:], n_classes=n_classes): 
    # Define model
    model = Sequential()
    model.add(layers.Convolution2D(16, (3, 3),
                            padding='same',
                            input_shape=input_shape, activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
    model.add(layers.MaxPooling2D(pool_size =(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(n_classes, activation='softmax')) 

    # Train model

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    print(model.summary())
    return model

model = define_model2()



In [ ]:



#https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
#es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', min_delta=1, verbose=1, patience=3)

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.n // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.n // batch_size,
    epochs = 100,
    callbacks=[es]
)

    